In [1]:
import sys
import os
sys.path.append(os.path.abspath('../'))

from evaluation import *
from data_loader import *
import random
from datetime import datetime

In [2]:
colors = {
    "DQN": 'darkorange',
    "PPO": 'forestgreen',
    "CMA": 'cyan',
    "ODT": 'blueviolet'
}

In [3]:
starting_exp = 4000
ending_exp = 4035
save_processed_data = True
#exp_lists_preformatted = ['4000_4035.csv', '4072_4107.csv'] # Existing formatted datasets
exp_lists_preformatted = []

target_agg_count = 6

exp_agent_data = []
exp_station_data = []

experiments = range(starting_exp, ending_exp + 1)

# REMOVE THIS
#algos = ['DQN', 'PPO', 'CMA', 'ODT']

if len(exp_lists_preformatted) == 0:
    for ind, exp_num in enumerate(experiments):
        config_fname = f'./Exp_{exp_num}/config.yaml'
        
        c = load_config_file(config_fname)
        nn_c = c['nn_hyperparameters']
        federated_c = c['federated_learning_settings']
        algo_c = c['algorithm_settings']
        env_c = c['environment_settings']
        eval_c = c['eval_config']
    
        if federated_c['aggregation_count'] != target_agg_count:
            print(f"Experiment {exp_num} does not have matching aggregation level")
            continue
        
        ev_info = []
    
        seed = env_c['seed']
    
        algorithm_dm = algo_c['algorithm']
        
        def load_from_json_with_error_handling(filepath, columns_specific):
            try:
                return read_csv_data(filepath, columns=columns_specific)
            except json.JSONDecodeError as e:
                print(f"Error decoding JSON from {filepath}: {e.msg} at line {e.lineno}, column {e.colno}")
                return None  # Handle the error and return None or an empty object
        
        
        d_base = f"../../../../storage_1/metrics/Exp_{exp_num}"
        
        if not os.path.exists(d_base):
            d_base = f"../metrics/Exp_{exp_num}"
                
        base_path = f"{d_base}/train/metrics"
    
        print(f'Loading {base_path}_agent_metrics.csv')
        agent_data = load_from_json_with_error_handling(f'{base_path}_agent_metrics.csv', ['episode', 'timestep', 'done', 'zone', 'aggregation', 'agent_index', 'distance', 'average_battery'])
        station_data = load_from_json_with_error_handling(f'{base_path}_station_metrics.csv', ['episode', 'timestep', 'done', 'zone', 'aggregation', 'station_index', 'traffic'])
        
        agent_data['seed'] = seed
        station_data['seed'] = seed
        
        agent_data['exp_num'] = exp_num
        station_data['exp_num'] = exp_num
        
        agent_data['algorithm'] = algorithm_dm
        station_data['algorithm'] = algorithm_dm
        #agent_data['algorithm'] = algos[ind % len(algos)] # REMOVE THIS AND UNCOMMENT PREVIOUS LINE!
        #station_data['algorithm'] = algos[ind % len(algos)] # REMOVE THIS AND UNCOMMENT PREVIOUS LINE!
        
        agent_data['season'] = env_c['season']
        station_data['season'] = env_c['season']
        
        agent_data['num_aggs'] = federated_c['aggregation_count']
        station_data['num_aggs'] = federated_c['aggregation_count']
        
        agent_data['eps_per_agg'] = nn_c['num_episodes']
        station_data['eps_per_agg'] = nn_c['num_episodes']
    
        exp_agent_data.append(agent_data)
        exp_station_data.append(station_data)

Experiment 4000 does not have matching aggregation level
Experiment 4001 does not have matching aggregation level
Experiment 4002 does not have matching aggregation level
Loading ../../../../storage_1/metrics/Exp_4003/train/metrics_agent_metrics.csv
Loading ../../../../storage_1/metrics/Exp_4004/train/metrics_agent_metrics.csv
Loading ../../../../storage_1/metrics/Exp_4005/train/metrics_agent_metrics.csv
Experiment 4006 does not have matching aggregation level
Experiment 4007 does not have matching aggregation level
Experiment 4008 does not have matching aggregation level
Experiment 4009 does not have matching aggregation level
Experiment 4010 does not have matching aggregation level
Experiment 4011 does not have matching aggregation level
Loading ../../../../storage_1/metrics/Exp_4012/train/metrics_agent_metrics.csv
Loading ../../../../storage_1/metrics/Exp_4013/train/metrics_agent_metrics.csv
Loading ../../../../storage_1/metrics/Exp_4014/train/metrics_agent_metrics.csv
Experiment 40

KeyboardInterrupt: 

In [ ]:
if len(exp_lists_preformatted) == 0:
    # Convert data to DataFrame for easier manipulation
    df_agent = pd.concat(exp_agent_data, ignore_index=True)
    df_station = pd.concat(exp_station_data, ignore_index=True)
    
    cumulative_agent_df = (
        df_agent
        .groupby(
            ['episode', 'zone', 'aggregation', 'agent_index', 'seed', 'exp_num', 'algorithm', 'season', 'num_aggs', 'eps_per_agg']
        )[['distance', 'average_battery']]
        .sum()
        .reset_index()
    )
    
    cumulative_station_df = (
        df_station
        .groupby(
            ['episode', 'zone', 'aggregation', 'station_index', 'seed', 'exp_num', 'algorithm', 'season', 'num_aggs', 'eps_per_agg']
        )['traffic']
        .max()
        .reset_index()
    )
    
    # Rename the 'reward' column to 'cumulative_reward' for clarity
    cumulative_agent_df.rename(columns={'average_battery': 'battery_used'}, inplace=True)
    cumulative_agent_df.rename(columns={'distance': 'distance_traveled'}, inplace=True)
    
    # Get recalculated episodes using (aggregation number * episodes per aggregation) + episode number
    cumulative_agent_df['episode'] = cumulative_agent_df['aggregation'] * cumulative_agent_df['eps_per_agg'] + cumulative_agent_df['episode']
    cumulative_station_df['episode'] = cumulative_station_df['aggregation'] * cumulative_station_df['eps_per_agg'] + cumulative_station_df['episode']
    
    cumulative_agent_df.head()

In [ ]:
if len(exp_lists_preformatted) == 0:
    cumulative_station_df.head()

In [ ]:
# Define the file path with starting_exp and ending_exp variables
file_path_for_processed_data = f'../../../../storage_1/metrics/formatted_experiment_data/part_4/'

if len(exp_lists_preformatted) == 0:

    # Get the data for the last episode only
    cumulative_agent_df = cumulative_agent_df[cumulative_agent_df['episode'] == cumulative_agent_df['episode'].max()]
    cumulative_station_df = cumulative_station_df[cumulative_station_df['episode'] == cumulative_station_df['episode'].max()]

    if save_processed_data:
        if not os.path.exists(file_path_for_processed_data):
            os.makedirs(file_path_for_processed_data)
        
        # Save the DataFrame to a CSV file
        cumulative_agent_df.to_csv(f"{file_path_for_processed_data}/agent_{starting_exp}_{ending_exp}.csv", index=False)
        cumulative_station_df.to_csv(f"{file_path_for_processed_data}/station_{starting_exp}_{ending_exp}.csv", index=False)
        
        print(f"Data successfully saved to {file_path_for_processed_data}/agent_{starting_exp}_{ending_exp}.csv")
        print(f"Data successfully saved to {file_path_for_processed_data}/station_{starting_exp}_{ending_exp}.csv")

if len(exp_lists_preformatted) > 0:
    agent_dataframes = []
    
    # Load and combine datasets
    for existing_set in exp_lists_preformatted:
        loaded_dataset = pd.read_csv(f"{file_path_for_processed_data}/agent_{existing_set}")
        # Combine datasets
        agent_dataframes.append(loaded_dataset)

    cumulative_agent_df = pd.concat(agent_dataframes, ignore_index=True)

    station_dataframes = []
    
    # Load and combine datasets
    for existing_set in exp_lists_preformatted:
        loaded_dataset = pd.read_csv(f"{file_path_for_processed_data}/station_{existing_set}")
        # Combine datasets
        station_dataframes.append(loaded_dataset)

    cumulative_station_df = pd.concat(station_dataframes, ignore_index=True)

In [ ]:
for algo in cumulative_agent_df['algorithm'].unique():
    print(f"===== Algorithm: {algo} =====")

    agent_d = cumulative_agent_df[cumulative_agent_df['algorithm'] == algo]
    station_d = cumulative_station_df[cumulative_station_df['algorithm'] == algo]

    print(f"\tAverage Traffic: {station_d['traffic'].mean():.2f}")
    print(f"\tAverage Distance Travelled: {agent_d['distance_traveled'].mean():.2f}")
    print(f"\tAverage Battery Used (kWh): {agent_d['battery_used'].mean():.2f}")
    print("\t--")
    print(f"\tTraffic Std Dev: {station_d['traffic'].std():.2f}")
    print(f"\tDistance Travelled Std Dev: {agent_d['distance_traveled'].std():.2f}")
    print(f"\tBattery Used Std Dev (kWh): {agent_d['battery_used'].std():.2f}")
    print("")

for season in cumulative_agent_df['season'].unique():
    season_data = cumulative_agent_df[cumulative_agent_df['season'] == season]
    
    for algo in season_data['algorithm'].unique():
        algo_data = season_data[season_data['algorithm'] == algo]

        avg_energy_used = algo_data['battery_used'].mean()
        std_energy_used = algo_data['battery_used'].std()

        print(f"Algorotim: {algo} - Season: {season} \
        - Avg. Energy {avg_energy_used:.2f} - Std. Energy {std_energy_used:.2f}")